# Annihilation Strain Function and Plot

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating new project at `~/bosonSR_repo`
new project at `~/bosonSR_repo`
  No Changes to `~/bosonSR_repo/Project.toml`
  No Changes to `~/bosonSR_repo/Manifest.toml`
  No Changes to `~/bosonSR_repo/Project.toml`
  No Changes to `~/bosonSR_repo/Manifest.toml`


In [2]:
Pkg.add(["SpecialFunctions", "CairoMakie"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving    Resolving package versions...
package versions...
   Installed Expat_jll ─────────── v2.7.3+0
   Installed oneTBB_jll ────────── v2022.0.0+1
   Installed StaticArraysCore ──── v1.4.4
   Installed IrrationalConstants ─ v0.2.6
   Installed StatsFuns ─────────── v1.5.2
   Installed Unitful ───────────── v1.25.1
   Installed StatsBase ─────────── v0.34.7
   Installed JSON ──────────────── v1.2.0
   Installed Expat_jll ─────────── v2.7.3+0
   Installed oneTBB_jll ────────── v2022.0.0+1
   Installed StaticArraysCore ──── v1.4.4
   Installed IrrationalConstants ─ v0.2.6
   Installed StatsFuns ─────────── v1.5.2
   Installed Unitful ───────────── v1.25.1
   Installed StatsBase ─────────── v0.34.7
   Installed JSON ──────────────── v1.2.0
   Installed ColorBrewer ───────── v0.4.2
   Installed StructArrays ──────── v0.7.2
   Installed Rmath ─────────────── v0.9.0
   Installed StructUtils ───────── v2.5.1
   Installed Col

In [ ]:
using SpecialFunctions, CairoMakie

# Requires omega in Hz, M in solar masses, alpha = dimensionless, r in kpc
# r is distance from us to binary

# H_ann gives the change in frequency from \omega_ann. Centered in the MHz, but with \delta f being extremely small.

const Mp = 1.220890e19
const G = 1 / Mp^2

function h_ann(omega, M, n, l, alpha, r)

    # Define called on functions
    function omega_ann(mua, alpha, n)
        return 2 * mua * (1 - alpha^2 / (2 * n^2))
    end

    function gamma_a(l, alpha, M)
        if l == 1
            p = 17
        else
            p = 4 * l + 1
        end
        rg = G * M
        return G * 1e-10 / rg^3 * (((alpha / l) * 0.5)^p + ((alpha / l) * 0.5)^(p + 1))
    end

    # Julia equivalent of mpmath exponential integral
    function _ei_vec(z)
        return expinti.(z) 
    end

    # Define Conversions and Constants
    Mp_local = 1.220890e19
    G_local = 1 / Mp_local^2
    M_sun = 2e30 * 5.61e26 # Solar mass -> GeV
    Mpc_to_Gev = 1.56e38        # 1 Mpc in GeV
    kpc_to_Gev = Mpc_to_Gev / 1000
    Hz_to_Gev = 4.1357e-24 # Hz -> GeV
    hbar_GeV = 6.582e-25
    
    # Convert to natural units
    M_ann = M * M_sun # Solar mass -> GeV
    mua = alpha / (G_local * M_ann) # Mu in GeV
    r_kpc = r * kpc_to_Gev # Kpc -> GeV
    println("Mua: ", mua)
    println("M: ", M_ann)

    omega_a = omega_ann(mua, alpha, n)
    Gamma = gamma_a(l, alpha, M_ann)

    # Approximate maximum population of states
    N_max = 10^76 * (M/10)^2
    omega_gev = omega .* Hz_to_Gev
    println("Omega (GeV): ", abs(minimum(omega_gev)))
    z = 1im * omega_gev / (Gamma * N_max)  # argument for Ei and exponential
    
    pref = 1/(2*π) * sqrt(4.0 * G_local / (Gamma * r_kpc^2 * omega_a))
    Ei = -2.0 * _ei_vec(z) - log.(-1im * omega_gev) + log.(1im * omega_gev) + 1im * π * sign.(real.(omega_gev))
    freq = omega_gev / (2*π*Hz_to_Gev)
    println("Freq: ", freq[1:5])  # Show first 5 values
    
    # frequency strain has units of h*GeV, need to convert to Hz and then divide by timescale of event (2e6) to match discrete fft
    return pref * exp.(z) .* Ei * hbar_GeV, freq
end

h_ann (generic function with 1 method)

In [24]:
import SpecialFunctions: expinti


z = 1 + 2im;

using SpecialFunctions

"""
    expinti(z::Complex)

Exponential integral expinti(z) for complex argument z, matching Python's mpmath.ei(z)
"""
function expinti(z::Complex)
    E1 = expint(1, -z)
    if imag(z) > 0
        return -E1 + im * π
    elseif imag(z) < 0
        return -E1 - im * π
    else
        # z real: approach from above
        return -E1
    end
end

expinti(z) 


1.0421677081649352 + 3.701501425937874im

In [23]:
expinti(z) + pi*im

1.0421677081649352 + 3.701501425937874im

In [18]:
using Pkg
Pkg.add("ExpIntegral")


Pkg.Types.PkgError: The following package names could not be resolved:
 * ExpIntegral (not found in project, manifest or registry)
[36m   Suggestions:[39m Int[0m[1me[22mgrals QCInt[0m[1me[22mgrals M[0m[1me[22mshIntegrals Simpl[0m[1me[22mIntegrals BibInt[0m[1me[22mrnal

In [12]:
expint(1, 1im) 

-0.3374039229009678 - 0.6247132564277136im

In [6]:
# Plot label params
title_fs = 20
label_fs = 20
tick_fs = 20
legend_fs = 15

# Relevant conversions
M_sun = 2e30 * 5.61e26 # Solar mass -> GeV

# Dependent Params
M_ann = 3.1e-04  # in solar masses
mua_ann = 2e-16  # in GeV
alpha_ann = G * M_ann * mua_ann * M_sun
r_kpc = 1
n = 4
m = l = n - 1

# call functions
omega = 10 .^ range(-13, -8, length=10000) # in Hz

ann_strain_result, frequency = h_ann(omega, M_ann, n, l, alpha_ann, r_kpc)
ann_strain_abs = abs.(ann_strain_result)

# Plot
fig = Figure()
ax = Axis(fig[1, 1],
    title="Annihilation Gravitational Wave Strain",
    xlabel="Δ Frequency [Hz]",
    ylabel="|h̃₊(f)|",
    xscale=log10,
    yscale=log10,
    grid=true,
    titlefontsize=title_fs,
    guidefontsize=label_fs,
    tickfontsize=tick_fs
)
lines!(ax, frequency, ann_strain_abs, color=:blue, label="Annihilation Strain")
display(ax)

Mua: 2.0000000000000002e-16
M: 3.4782e53
Omega (GeV): 4.1357e-37

M: 3.4782e53
Omega (GeV): 4.1357e-37


MethodError: MethodError: no method matching expinti(::ComplexF64)
The function `expinti` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  expinti(!Matched::BigFloat)
   @ SpecialFunctions ~/.julia/packages/SpecialFunctions/6nmlv/src/expint.jl:608
  expinti(!Matched::Float16)
   @ SpecialFunctions ~/.julia/packages/SpecialFunctions/6nmlv/src/expint.jl:606
  expinti(!Matched::Float32)
   @ SpecialFunctions ~/.julia/packages/SpecialFunctions/6nmlv/src/expint.jl:605
  ...


# Binary Merger Template and Plot
check this code with Henry

In [ ]:
function htilde_plus(
    f,                     # array-like (Hz in your chosen units)
    Msun, r, alpha,        # source mass (Solar mass), distance (kpc), fine-structure parameter,  
    q,                     # binary mass ratio (companion/host)
    m_i, m_f,              # initial/final magnetic quantum numbers (e.g., 1 -> -1)
    eta;                   # parameter entering z (or leave None to use z_scaling below)
    use_z_scaling=false,   # if True, use Eq. (15.5) instead of z=η²/(|Δm|γ)
    relativistic=false
)

    # Horizon Angular velocity function, takes spin param a* and r_plus spin horizon radius. Units are in inverse GeV
    function Omega_plus(a, r)
        return a / (2 * r)
    end

    # Omega needs to be in units of GeV. Since mass is only thing dimensional, 
    # need to add a factor of G into denominator ( [1/(M*G)] = GeV )
    function Omega0_hyp(M, m_i, alpha, n, l)
        numerator = 64 * m_i * alpha^7
        denominator = G*M * n^3 * (2^l) * (2*l + 1) * (2*l + 2) * (m_i^2 + 4 * alpha^2)
        return numerator / denominator
    end

    # Kerr spin horizon radius, takes Mass in GeV, and dimensionless spin
    function r_plus(M, a)
        return M * (1 + sqrt(1 - a^2)) * G
    end
    
    # Superradiant rate
    function super_gamma(n, l, m, omega, mu, M, r, a)

        function glm(a, m, l, r, omega)
            g = 1
            if l == 1
                g *= l^2 * (1 - a^2) + (a * m - 2 * r * omega)^2
            end
            if l != 1
                for k in 1:(l-1)
                    g *= k^2 * (1 - a^2) + (a * m - 2 * r * omega)^2
                end
            end
            return g
        end

        C = (
            2^(4 * l - 1)
            * factorial(n + l)
            / (n^(2 * l + 4) * factorial(n - l - 1))
            * (factorial(l) / (factorial(2 * l) * factorial(2 * l + 1)))^2
        )
        g = glm(a, m, l, r, omega)
        omega_plus = Omega_plus(a, r)
        gamma_nlm = (
            2 * r / M * C * g * (m * omega_plus - omega) * (G * mu * M)^(4 * l + 5) / G 
        )
        return gamma_nlm
    end

    # takes dimensionless quantum number m_i and alpha
    function a_tilde_crit(m_i, alpha)
        m2 = m_i^2
        return (4.0 * m_i * alpha) / (m2 + 4.0 * alpha^2)
    end

    # all dimensionless
    function q_c(alpha, m_i, a_tilde)
        m = float(m_i)
        one_minus = (1.0 - a_tilde / m)
        # guard small rounding errors inside the sqrt:
        inner = 1.0 - (4.0 * alpha / m * one_minus)^2
        inner = clamp(inner, 0.0, 1.0)

        denom = m^2 * (1.0 - sqrt(inner))^2
        # Avoid divide-by-zero:
        if denom == 0.0
            return 0.0
        end

        return 8.0 * alpha^2 * one_minus / denom
    end

    # M is host BH mass
    # takes mass in GeV, dimensionless q_c, r in GeV is distance from Earth, Omega_0 is angular momentum at BH horizon in GeV
    function h0_from_params(qc, M, r, alpha, Omega0)
        return 24.0 * G*(qc * M / r) * (alpha^(-4)) * (G*M * Omega0)^2
    end

    function gamma_rate(q, M, Omega0)
        return (Omega0^2) * (96.0 / 5.0) * (q / (q + 1.0)^(1.0 / 3.0)) * (M * Omega0)^(5.0 / 3.0)
    end

    function z_parameter(eta, Delta_m, gamma)
        return (eta^2) / (abs(Delta_m) * gamma)
    end

    function z_scaling_211_to_21m1(alpha, q)
        """
        z_{211->21-1} ≈ 7 * (1.81/(1+4 α²))^(1/3) * (q)^(1/3) * (2/(1+q))^(5/3) * (0.45/α)^(11/3)
        """
        return 7.0 * (1.81 / (1.0 + 4.0 * alpha^2))^(1.0 / 3.0) *
                (q^(1.0 / 3.0)) *
                (2.0 / (1.0 + q))^(5.0 / 3.0) *
                (0.45 / alpha)^(11.0 / 3.0)
    end

    function fc_from_Omega0(Omega0)
        return (2.0 / π) * Omega0 * 1e23
    end

    function psi_plus(f, r, f0, Delta_m, gamma)
        return f .* r .+ (f .- f0).^2 ./ (4.0 * abs(Delta_m) * gamma) .- π / 4.0
    end

    # New Function used in the adjusted amplitude from Henry's paper
    function g_func(z)
        z = complex.(z)
        num = exp.(-π * z .+ 2 * z .* atan.(2 * z))
        den = sqrt.(1 .+ 4 * z.^2)
        return num ./ den
    end

    # Relevant constants/conversions
    Mp_local = 1.220890e19
    G_local = 1 / Mp_local^2   
    M_sun = 2e30 * 5.61e26 # Solar mass -> GeV
    r *= 1.6e35 # Kpc -> GeV^-1
    # Convert to natural units
    M = Msun * M_sun # Solar mass -> GeV
    m_boson = alpha / (G_local * M) # Boson mass in GeV

    a = a_tilde_crit(m_i, alpha)
    Omega0 = Omega_plus(a, r_plus(M, a)) / (2.0/π) 
    Omega_binary = Omega0_hyp(M, m_i, alpha, 2, 1)

    println("Omega Binary: ", Omega_binary)
    println("Omega BH: ", Omega0)
    
    # gamma takes BH rotational freq
    if relativistic == false
        Gamma_abs = super_gamma(2, 1, -1, Omega0, m_boson, M, r, a) 
    else
        # Replace with relativistic superradiance rate
        Gamma_abs = 0 
    end

    f = float.(f)
    Delta_m = abs(m_f - m_i)

    # ã_crit and q_c
    acrit = a_tilde_crit(m_i, alpha)
    qc = q_c(alpha, m_i, acrit)

    # h0 amplitude (15.12)
    h0 = h0_from_params(qc, M, r, alpha, Omega_binary)

    gamma = gamma_rate(q, M, Omega_binary)

    # z (15.1) or scaling (15.5)
    if use_z_scaling
        z = z_scaling_211_to_21m1(alpha, q)
    else
        z = z_parameter(eta, Delta_m, gamma)
    end

    # central frequency and phase (under 15.15)
    f_c = fc_from_Omega0(Omega_binary)
    f0 = f_c
    phase = psi_plus(f, r, f0, Delta_m, gamma)

    # denominator and envelope
    denom = (sqrt(z) / abs(Gamma_abs)) .- 1im * π * (f .- f_c)
    envelope = exp.(-π * z) .* exp.(-2.0 * z * atan.(π * (f .- f0) / abs(Gamma_abs)))

    # Fix: include actual inclination:
    function with_inclination(iota)
        pref = h0 * (1.0 + cos(iota)^2) * sqrt(π) * (Delta_m^2)
        return sqrt(G_local) * pref * exp.(1im * phase) .* envelope ./ denom
    end

    return with_inclination
end

# Given h_+(f) and inclination iota [rad], return h_x(f)
function hcross_from_hplus_freq(hplus_f, iota)
    return 1im * (2*cos(iota)/(1+cos(iota)^2)) * hplus_f
end

In [ ]:
# Relevant Input parameters, units defined
Mp_local = 1.220890e19
G_local = 1 / Mp_local^2

M = 1e3      # BH mass in solar masses
r = 1        # kpc distance away
alpha = 0.3
m_boson_param = alpha/(G_local*M) # Boson mass in GeV
q = 0.1      # Binary mass ratio
m_i = 1
m_f = -1     # Quantum Numbers
eta = 5.0e-2 # parameter entering z (or leave None to use z_scaling below)

# Found for this freq range, can also be estimated using Omega0 function 
f = range(6e-4, 4e-3, length=10000)

hplus_of_iota = htilde_plus(f, M, r, alpha, q, m_i, m_f, eta, use_z_scaling=false, relativistic=false)

iota = π  # rad
Hplus = hplus_of_iota(iota)
Hcross = hcross_from_hplus_freq(Hplus, iota)

# --- Plot h+ ---
p2 = plot(f, abs.(Hplus),
    xlabel="Frequency f (Hz)",
    ylabel="|h̃₊(f)|",
    title="Binary Perturbation — Plus Polarization",
    xscale=:log10,
    yscale=:log10,
    grid=true,
    linewidth=2,
    size=(800, 600),
    titlefont=font(title_fs),
    guidefont=font(label_fs),
    tickfont=font(tick_fs)
)
display(p2)

# --- Plot h× ---
p3 = plot(f, abs.(Hcross),
    xlabel="Frequency f (Hz)",
    ylabel="|h̃ₓ(f)|",
    title="Binary Perturbation — Cross Polarization",
    xscale=:log10,
    yscale=:log10,
    grid=true,
    linewidth=2,
    size=(800, 600),
    titlefont=font(title_fs),
    guidefont=font(label_fs),
    tickfont=font(tick_fs)
)
display(p3)